### Calculating various paramters for cell object, storing them into CSV files and objects themselves

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import skimage
from scipy.ndimage import center_of_mass
from scipy.signal import correlate
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from skimage.measure import label, regionprops, regionprops_table
from scipy import ndimage
import pickle
import tifffile
import pandas as pd
import skimage.io
import skimage.color
import skimage.filters
from scipy.stats import pearsonr
from cell_obj import CellObj, add_measured_value, save_objects_as_pickle

In [2]:
def show_cell_and_mask(cell_obj):
    '''shows induvidual cell from a cell object'''
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(cell_obj.images_dict['cut_image'], interpolation='nearest', cmap=plt.cm.gray)
    ax2.imshow(cell_obj.images_dict['mask'], interpolation='nearest', cmap=plt.cm.gray)
    plt.show()

In [3]:
def calculate_morpho_properties(cell_obj, properties):
    '''returns a dict of a given moprhological measurments from skimage.measure'''
    props = regionprops_table(cell_obj.images_dict['mask'].astype('int'), properties=properties)
    return_dict = {}
    for key, value in props.items():
        return_dict[key] = float(value)
    return return_dict

In [4]:
def calculate_intensity(cell_obj, use_mask=False):
    # Can be used as a baseline for other functions
    '''calculate intensity and std for all channels based on mask '''
    image = cell_obj.images_dict['cut_image']
    #if there is no mask the whole image is masked
    if use_mask:
        mask = cell_obj.images_dict['mask']
        mask = np.invert(mask)
    else:
        mask = np.ones_like(image).astype(bool)
        mask = np.invert(mask)
        
    mean_std_dict = {}
    return_dict = {}
    
    masked_image = np.ma.array(image, mask = mask)
    image_mean = round(np.mean(masked_image),2)
    image_std = round(np.std(masked_image),2)
        
    mean_std_dict['mean_intensity'] = image_mean
    mean_std_dict['mean_std'] = image_std
        
    for key, value in mean_std_dict.items():
        return_dict[key] = float(value)
    return return_dict

In [5]:
def calculate_intensity_per_channel(cell_obj, use_mask=False):
    # Rewrite or ditch
    '''calculate intensity and std for each channel based on mask '''
    image = cell_obj.images_dict['cut_image']
    #if there is no mask the whole image is masked
    if use_mask:
        mask = cell_obj.images_dict['mask']
        mask = np.invert(mask)
    else:
        mask = np.ones_like(image[...,0]).astype(bool)
        mask = np.invert(mask)
        
    mean_std_dict = {}
    return_dict = {}
    
    # iterates over channels for masking
    for channel in range(image.shape[-1]):
        masked_image = np.ma.array(image[...,channel], mask = mask)
        channel_mean = round(np.mean(masked_image),2)
        channel_std = round(np.std(masked_image), 2)
        
        mean_std_dict[f'mean_channel_{channel}'] = channel_mean
        mean_std_dict[f'std_channel_{channel}'] = channel_std
        
    for key, value in mean_std_dict.items():
        return_dict[key] = float(value)
    return return_dict

In [6]:
def calculate_colocalization(cell_obj, channels, use_mask=False):
    '''calculatew Pearson product-moment correlation coefficients for two channels '''
    image = cell_obj.images_dict['cut_image']
    #if there is no mask the whole image is masked
    
    if use_mask:
        mask = cell_obj.images_dict['mask']
        mask = np.invert(mask)
    else:
        mask = np.ones_like(image[...,0]).astype(bool)
        mask = np.invert(mask)
        
    corr_dict = {}
    return_dict = {}
    
    image_channel_1 = image[...,channels[0]]
    image_channel_2 = image[...,channels[1]]

    masked_image_channel_1 = np.ma.array(image_channel_1, mask = mask)
    masked_image_channel_2 = np.ma.array(image_channel_2, mask = mask)
  

    p_correlation = pearsonr(masked_image_channel_1.ravel(), masked_image_channel_2.ravel())

    corr_dict['cor_coefficient'] = p_correlation[0] 
    corr_dict['p_value'] = p_correlation[1] 
        
    for key, value in corr_dict.items():
        return_dict[key] = round(float(value),2)
        
    return return_dict

In [7]:
def calc_center_of_mass(cell_obj):
    '''returns center of mass for a given mask for an original image'''
    bounduing_box_x = cell_obj.box_coord[2]
    bounduing_box_y = cell_obj.box_coord[0]
    
    center_of_mass_coords =  center_of_mass(cell_obj.images_dict['mask'].astype('int'), labels=None, index=None)
    return_dict = {}
    return_dict['center_of_mass_x'] = int(bounduing_box_x + int(center_of_mass_coords[0]))
    return_dict['center_of_mass_y'] = int(bounduing_box_y + int(center_of_mass_coords[1]))
    return return_dict

In [8]:
def add_measured_value(cell_obj_list, function, **kwargs):
    '''Adds calculated measurments to every cell object in a list usong functions'''
    new_cell_obj_list = []
    
    for cell_obj in cell_obj_list:
        props = function(cell_obj, **kwargs)
        # accepts return as a dictionary, iterates over dictionary and adds it to value dictionary
        for key, value in props.items():
            cell_obj.measured_values[key] = value
        new_cell_obj_list.append(cell_obj)
        
    return new_cell_obj_list

In [9]:
def create_df(cell_list):
    # get a list of all measured_values names
    column_names = [key for key in cell_list[0].measured_values.keys()]
    cell_objects_data_dict = {}
    
    for count, cell_obj in enumerate(cell_list):
        values_list = []
        for value in cell_obj.measured_values.values():
            values_list.append(value)
        cell_objects_data_dict[count] = values_list 
        
    df = pd.DataFrame(data=cell_objects_data_dict).transpose()
    df.columns = column_names
    
    return df     

In [10]:
cell_object_list_path = 'cell_lists/cell_list.pickle'
cell_list = []
#Opening file and unpuckling its content
with open(cell_object_list_path , 'rb') as pickled_file:
    cell_list = pickle.load(pickled_file)

In [11]:
#Channels to compute correlation between
channels = [0,2]

In [12]:
# Sequantial calculating data for cell objects
cell_list = add_measured_value(cell_list, calculate_morpho_properties,properties=('feret_diameter_max',
                                                 'perimeter',
                                                 'eccentricity'))
cell_list = add_measured_value(cell_list, calc_center_of_mass)
cell_list = add_measured_value(cell_list, calculate_intensity)
cell_list = add_measured_value(cell_list, calculate_colocalization, channels=channels, use_mask=True)
#cell_list = add_measured_value(cell_list, calculate_intensity_per_channel)

In [13]:
save_objects_as_pickle(cell_list, 'cell_list_for_labeling')

In [14]:
df = create_df(cell_list)

In [15]:
df.head()

,feret_diameter_max,perimeter,eccentricity,center_of_mass_x,center_of_mass_y,mean_intensity,mean_std,cor_coefficient,p_value
0,22.847319,65.698485,0.426620,126.0,13.0,34.23,56.28,0.44,0.0
1,21.540659,60.284271,0.488431,172.0,13.0,31.83,51.43,0.51,0.0
2,30.413813,86.426407,0.565046,233.0,18.0,33.47,53.88,0.39,0.0
3,22.135944,63.112698,0.498589,36.0,21.0,27.18,44.48,0.39,0.0
4,23.259407,64.526912,0.573751,74.0,34.0,27.22,45.06,0.45,0.0


In [16]:
df.to_csv(f'csvs/{cell_list[0].original_image_hash}.csv', index=False)  